In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, ParameterGrid
from sklearn.metrics import accuracy_score

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

from bayesian_opt import ThompsonSampling,GP_UCB
# try to not change any code from above file!!

import warnings
warnings.filterwarnings('ignore')

# toy datasets: wine, breast_cancer, diabetes
# real datasets: california_housing
# synthetic datasets: make_classification (select hardness level yourself)

# try logistic regression and random forest
# compare with grid search (performance, time, total trials)

In [2]:
data = load_wine()

X = data['data']
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scaler = StandardScaler()

kf = KFold(n_splits=5)

### Bayesian HPO

In [3]:
param_grid = {'C': np.logspace(-4, 4, 20), 'l1_ratio': np.arange(0,1.01,0.1)}
hyper_matrix = np.array(list(map(lambda x: list(x.values()),list(ParameterGrid(param_grid)))))

hyper_scaler = StandardScaler()

hyper_matrix = hyper_scaler.fit_transform(hyper_matrix)


In [4]:
def compute_cv_score(temp_hyp_cfg):

    temp_hyp_cfg = hyper_scaler.inverse_transform(temp_hyp_cfg)

    scores = []
    for train_index, test_index in kf.split(X_train):   
        X_train_cv, X_valid_cv = X_train[train_index], X_train[test_index]
        y_train_cv, y_valid_cv = y_train[train_index], y_train[test_index]

        X_train_cv = scaler.fit_transform(X_train_cv)
        X_valid_cv = scaler.transform(X_valid_cv)

        model = LogisticRegression(solver = 'saga', penalty='elasticnet',C = temp_hyp_cfg[0], l1_ratio = temp_hyp_cfg[1])

        model.fit(X_train_cv,y_train_cv)

        y_pred_cv = model.predict(X_valid_cv)
        scores.append(accuracy_score(y_valid_cv,y_pred_cv))

    print(f'hyper_config: {temp_hyp_cfg}, mean: {np.mean(scores):.2f}, std: {np.std(scores):.2f}')
    return np.mean(scores)


kernel = 0.8 * RBF(0.5,length_scale_bounds="fixed")
gpr = GaussianProcessRegressor(kernel=kernel)

ucb = GP_UCB(gpr,hyper_matrix,compute_cv_score,beta = 2.0)

tol = 5

for i in range(tol):
    
    ucb.query_new_point()

    ucb.update_gp()



hyper_config: [1.e-04 0.e+00], mean: 0.40, std: 0.09
hyper_config: [78.47599704  0.2       ], mean: 0.98, std: 0.02
hyper_config: [1.43844989e+03 2.00000000e-01], mean: 0.98, std: 0.02
hyper_config: [5.45559478e+02 4.00000000e-01], mean: 0.97, std: 0.02
hyper_config: [1.43844989e+03 4.00000000e-01], mean: 0.98, std: 0.02


### Grid Search

In [ ]:
# for each hyper config
param_grid = {'C': np.logspace(-4, 4, 20), 'l1_ratio': np.arange(0,1.01,0.1)}

for hyp_cfg in list(ParameterGrid(param_grid)):

    scores = []
    for train_index, test_index in kf.split(X_train):   
        X_train_cv, X_valid_cv = X_train[train_index], X_train[test_index]
        y_train_cv, y_valid_cv = y_train[train_index], y_train[test_index]

        X_train_cv = scaler.fit_transform(X_train_cv)
        X_valid_cv = scaler.transform(X_valid_cv)

        model = LogisticRegression(solver = 'saga', penalty='elasticnet',C = hyp_cfg['C'], l1_ratio= hyp_cfg['l1_ratio'])

        model.fit(X_train_cv,y_train_cv)

        y_pred_cv = model.predict(X_valid_cv)
        scores.append(accuracy_score(y_valid_cv,y_pred_cv))

    print(f'hyper_config: {hyp_cfg}, mean: {np.mean(scores):.2f}, std: {np.std(scores):.2f}')